In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [4]:
import os
from Bio import SeqIO
import subprocess

family_group = "7.Papilionidae"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "16":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Baronia_brevicornis
GCA_031353405.1_ASM3135340v1_genomic.fna
29111 29207
0 1
Getting Gene
MEWVLRSPQIHSTFGNLGYLHADGYHVNENCN
>Baronia_brevicornis_Exon_1_JASFAT010000114.1_29111_29207_left_0_right_1
ATGGAATGGGTTCTTAGGAGTCCTCAAATCCACAGCACTTTTGGCAATCTTGGATACCTACATGCAGATGGATATCACGTAAATGAAAACTGTAATG
29298 29848
2 0
Getting Gene
>Baronia_brevicornis_Exon_2_JASFAT010000114.1_29298_29848_left_2_right_0
cTGCCTTagaaactatattaaataatatattaacagaagacaaatatttatgtaactaTAGAAGAAGTATTAGTTTCGGGCAAAACATCAAAAAAGACCTCATACCTTTATTGATTAATGCAAAGGAACAAAAAACAGTAGAATTGTTAATCAAAATACTTGTGAATTTAACTATACCGATCGAATGTTTACTTTCACTGGATGCAATttcgaaaactgattttggaaGGCATACAATTTTTGAAATCAACAATCTCCTCGCATCTACCAAAAACTCTTTGACAGACCATAGAGCAACGAAAGCTGTTATCGAATTCTTGAAGAAAAATGCTGAAGCcgtacaaaatacaaaattatccaCTGAACAAATCACAAATATTAGCAATACGCTTTTATTTCTGcgcaatatattacatatacccGAAAATGTTAACTCTGTTAGCCCAAATTACAATGGGTCTTCTCATACTGtgcaaaatcaaattttatggAACTTATTTAGTCATAACATCGATAAAGTTCTTATCAAGTTGATGGGTATGCCTGATGCG
30028 30181
0 1
Getting Gene
>Baronia_brevicornis_Ex

8507976 8508076
0 2
Getting Gene
>Battus_philenor_Exon_8_JAQMYT010000002.1_8507976_8508076_left_0_right_2
GACCTCATAGTCACCAACCACATCTTTTTGATGTTTGTGGATACTGTTATGAAATTGCCAGAATACAAAGGTTCTACTAATATGGCCGATCACATTAAACA
8508155 8508353
1 0
Getting Gene
>Battus_philenor_Exon_9_JAQMYT010000002.1_8508155_8508353_left_1_right_0
GTTTGCTACACCTGAAATAATGTCACAGTACGGTCTTCTTCTTGAAGATTATGAAGGTAATGGAGGGTTTGTGAACGATTGCATTTTTACGATGATGCATCACGTCGGTGGAGAGTTGGGTTGTGTCATAGCCCTGTTTCAaccgaaaattttaaaaacttttacttctATCTGGAAATCTGAATTTGAAACCTGCGAC
8508630 8508811
0 2
Getting Gene
>Battus_philenor_Exon_10_JAQMYT010000002.1_8508630_8508811_left_0_right_2
gattGGGCTGATCTTATTGAGCATgttatcaatacatttattaaaaaacctcaAACATTGCAAAGCGTCGGCAACTTTCGATTGGATCCGGAGAACTTTGAGATGGCACAAGATGAAGAAGAAGGCGAAACACCGGCCAAGAGAAAATCAACTACTAGCAGTGTCACTAACACTAAAGAAaa
8509477 8509961
1 1
Getting Gene
>Battus_philenor_Exon_11_JAQMYT010000002.1_8509477_8509961_left_1_right_1
ATGGACTGAAGATGAAACTTCGTCTCTGAGCTGGTACTACATGCAGTGTAACACTACGTGTGATGTAATCGGAGAAGtagtaaagttat

Ornithoptera_alexandrae
GCA_029030555.1_ASM2903055v1_genomic.fna
7364372 7364468
0 1
Getting Gene
MEWVLRSPQLHNTFSSLGFVHSDGYHINEHCN
>Ornithoptera_alexandrae_Exon_1_JARDVY010000027.1_7364372_7364468_left_0_right_1
ATGGAGTGGGTCCTAAGAAGCCCTCAGCTACATAACACATTTAGTAGCCTAGGATTTGTCCATTCAGATGGATATCACATTAATGAACACTGCAATG
7362994 7363544
2 0
Getting Gene
>Ornithoptera_alexandrae_Exon_2_JARDVY010000027.1_7362994_7363544_left_2_right_0
CTGCCTTGGAAACaattttgcaaaatattttaaatgaagatAAATACTTATGTACTTATCGTAGAAGTATTAGTTTCGGCCAAAACATCAAAAAAGATCTCATACCGTTATTGATAAACACAAAAGACGATAAAACAATAGAGCTGTTGATAAAGATATTGGTCAACCTAACTATACCGTTAGAGTGTTTGTTGTCAGTGGAAGCAATTTCTAAAACTGACTTTGGCAGGCATGcgatatttgaaattaagaaTTTACTTGCTTCTACCAAACTTGCTCTAACTGATAACCGCGCGACAAAGGCGATAACAGATTTTCTCAAAAAAAATGCAGATgtagagaaaaaaacaaaactatccCCTGAGcgatgtgcaaatattagtaaCACACTTTTATTATTGCGAAACATCCTGCATATACCcgaggaaataaataatgttagtcCTTCTTATAACGGCACCTCCCACACTGtacaaaatcaaatattatggAATCTTTTTAGTCAGAATATTGACAAAATTCTCATAAAACTGATGGGTGTACCTGAAGCG
7362701 7362854
0 1
Gett

>Ornithoptera_priamus_Exon_7_JARDWD010000611.1_1247226_1247417_left_0_right_0
GTCGTTACCGCCATTAGAGAATTCGTTCAAGCGATCGAGCTATATAAGAAATCGCCTCATACGTGTGAAGATGATCAGGAGAAGCTAATTGAACTTCAGATGAAAATGTGTGAAACAGAGGAATTGCGTTCGCttttagttttacttttaagaCACTACAATCCGaaatatcatcctaaacaatatttgcag
1245866 1245966
0 2
Getting Gene
>Ornithoptera_priamus_Exon_8_JARDWD010000611.1_1245866_1245966_left_0_right_2
GATCTTATTGTCACAAACCACATTTTCTTGATGTTTGTTGATAGTGTAATGAAAATTCCGGAATATAAGGGTTCGACTAGTATGGTTGATCACATAAAACA
1245587 1245785
1 0
Getting Gene
>Ornithoptera_priamus_Exon_9_JARDWD010000611.1_1245587_1245785_left_1_right_0
atTCGCTACACCGGAAATAATGTCACAGTATGGACTTCTACTTGAAGATTATGAAGGCAATGGAGAGTTTGTTAATGATTGTATATTCACCATGATGCATCACGTAGGTGGAGAGTTGGAATGCGTCATAGCTTTATTTCAaccaaagattttaaaaacattcacCTCAATATGGAAATCGGAGTTTGAAACATGCGAC
1245136 1245314
0 2
Getting Gene
>Ornithoptera_priamus_Exon_10_JARDWD010000611.1_1245136_1245314_left_0_right_2
GATTGGTCTGATCTAATAGAGCACGTTATAAatacgtttataaaaaaaccacAAACGTTGCAAAGCGTTGGTAACTATCGACTAGAATCGGAA

>Papilio_dardanus_tibullus_Exon_15_QDHC01002768.1_191457_191669_left_1_right_2
TGCTATTGTCAAGGAAGAGATAATCGAAGTTAAGGTAACAGAGCTGGAGGATAGGGCGGGTCAGGAGGGGATGCGCGAGGGGCAGGGCGCGGGGCCGGGCGCGGCGGAGGAGGACGCGGTGGTCGACATGCCCGCCGCCATGAGCGCCCTCAGCGGCAAGCCCGAGATCAACTACAACTTGCACCACATGGAGTCTGAATATTACAACAACAG
191950 192130
1 0
Getting Gene
CGTGTGCGAGACGGCGTCGGTGGCGTCGGACCTGACGCGCATGTACGTGTCGGACGAGGAGGAGAAGCCGGAGCCGCGGCCCGTCAAGGCGCTCAGCGACGAGCTCTCGTGCAGCGACACGGACAGCGTCACACCCAGCGCGGAGAAAAGACCTAGAGTCGCTTTCTTTCCTACCTATTGA
>Papilio_dardanus_tibullus_Exon_16_QDHC01002768.1_191950_192130_left_1_right_0
CGTGTGCGAGACGGCGTCGGTGGCGTCGGACCTGACGCGCATGTACGTGTCGGACGAGGAGGAGAAGCCGGAGCCGCGGCCCGTCAAGGCGCTCAGCGACGAGCTCTCGTGCAGCGACACGGACAGCGTCACACCCAGCGCGGAGAAAAGACCTAGAGTCGCTTTCTTTCCTACCTATTGA
Papilio_demoleus
GCA_029286655.1_kiz_Pde_v1_genomic.fna
6093151 6093247
0 1
Getting Gene
MEWVLRSPQLHSTFSNLGFLHSDGYHVNEHCN
>Papilio_demoleus_Exon_1_JAGSMY010000006.1_6093151_6093247_left_0_right_1
ATGGAATGGGTCCTCAGGAGTCCTCAGCTTCACAGTA

...

>Papilio_memnon_Exon_13_LD699455.1_4226728_4226905_left_2_right_2
ctaaattaaagttttcgaTGAGTGAAGTAATGGAGACTAGTGTGTCGAAGTCGAATCAGAGAAACTCTTCAGCGATGACGCCGAGCACAAAAGACGCGATGATTACTACTCCTCTTGATAATTTCTACAAAGTTAACAAACAACAGCACTTTGGTACCGCCGTTAACTTCACCCCtat
4227024 4227128
1 2
Getting Gene
>Papilio_memnon_Exon_14_LD699455.1_4227024_4227128_left_1_right_2
gcCTGGCACGTCATTCAGTTTGAACGAAGACGAATTTCAAAAGCACAATTGGCTCGAAGTAGTTCAGAAATCTCAAGAATATAAGATATCTTTTGATCTTGAAGG
4227282 4227521
1 2
Getting Gene
>Papilio_memnon_Exon_15_LD699455.1_4227282_4227521_left_1_right_2
CACTATTCTCGAGGAAGAGAAAATCGAAGTGAAGGTGACAGAGGTGGAGAGCAAGTCGGGACAAAAGAGGGTGCGCGAGGTACTGTGCAGGGCGCGGGTGCTGGAGGAAGTGGCGCCGGAGGGGATCGTGGCCTCGCCGGGCGCGGCGCTGGTAATGCCCACCGCCATGGATGTCATGAGCGCTAAGCCCGACATCAATTACAACTTGCACCACTTGGAGTCTGAATATTTCAATACCAG
4228423 4228618
1 0
Getting Gene
CGTGTGCGAGACGGCGTCGGTGGCATCGGACCTGACGCGCATGTACGTGTCGGACGAGGAGGAGAAACCGGAGCCGCTGCGGCAGACGCGCGAGCTCGCGGAGAAGGCCGGCGACGATCTGTCGTTCAGCGACACCGACAGCATTGCGCCCGTCGTCGAGAAAAGAGCCAGAGTCAC

>Papilio_xuthus_Exon_4_NW_013533994.1_1080362_1080681_left_2_right_0
GTAGTAAGGATTCTTCGCCAAATTTGACTTCCGATCCAACTTCTGATTCGTCGGACACCGGCGGAAGCGGCAAAAGTAACGACGACCCCAGCTCTGTGAACAACGACTGGGATGCTTCTTCAAGAAACAACATGAACAGTAGCCAACCATTCCAGTCCCCGCGATCCGCCGACAATCACGCATCTGAAacaatctataataataatagcggCGATAGCGGTGATGGCGGCGATGGTGGTGATGGTGGTGATGGCGGTGATGGTGGCGATGGTGTTGATGGTgttgataataataaagataccACTATGAAAAATGATACG
1080000 1080286
0 2
Getting Gene
>Papilio_xuthus_Exon_5_NW_013533994.1_1080000_1080286_left_0_right_2
cTGGTTTCCGTGTCAGGTTCAAATGAAAACGATAGTCAAAATTATCAACAAAGAAAATCAGAAAAGAAAGTGGTCATATCAGAAAACTCTGATTACGGTTACGTCACTCAAATCGAAAATCAGGAGTCGATATCCACATCGAGCAACGAAGATGAGTTGCCAACTAAAAAACCTCAGCACCAAAAACCGCACACGACCAAACAGAGAATAAGTAACAAAGTACGCTCGAACCTCACGCTTCaggaaagaaaaagaaaaaaaatcgtcaaGAAAGGAAGAATAAATAT
1079568 1079922
1 0
Getting Gene
>Papilio_xuthus_Exon_6_NW_013533994.1_1079568_1079922_left_1_right_0
aacaaCACTGCAAGGACACAATACACTGACCGACGACGACATTTCCAATGTTTTGAAAGAATTTACAGTTAATTTTCTGTTAAAAGGATATAATTCATTAGTGAAGACGTTACATAAA

12087420 12087620
2 1
Getting Gene
>Sericinus_montela_Exon_12_JAGSMR010000012.1_12087420_12087620_left_2_right_1
ttttaagtCAGGCTGTGCCGCTGGTGCCATGGAACTGTGAACAAGCAAACATCTGTAAGGACTTAAAATTCCTGCAGCTTTTACATAAGCTTGGTTTTCATATGCCTGTTGATTCCGGTAAAGTATTCATTAGAATACCTCATTCTTGGACGACTGATATGATATTTGATGTCGCTAGCAAAATATCACCTATAAATAAAG
12087173 12087347
2 2
Getting Gene
>Sericinus_montela_Exon_13_JAGSMR010000012.1_12087173_12087347_left_2_right_2
CCAAATTGAAATTCTCCATCAAACAAGTCATGGGGACCAGTGCTGAAAAGTCTGAAATGTCACAAGCCGCGCCCTCATCAGAATCAACAGATGTCTCTATGACAACTGTCGACAAACACTTCTACCAAATAAATAAGCAGAAACATCTTGGAGCGCTTGTTAATTACACACCTAT
12086255 12086359
1 2
Getting Gene
>Sericinus_montela_Exon_14_JAGSMR010000012.1_12086255_12086359_left_1_right_2
GCCtggaattatatttaaaacgcCGAACGAAGAAATTCCGCTGCCGAATTGGCTTAAATTGATCCAAAAATCTCAAGAGCATAATATAAAATTCGATTTGGAAag
12085000 12085179
1 2
Getting Gene
>Sericinus_montela_Exon_15_JAGSMR010000012.1_12085000_12085179_left_1_right_2
CTCAAACATCAAGGAGGAATCATCGGATGTCAGTATGACAATAACTGAAGTGGCGGAAACTGAC

7702074 7702327
2 0
Getting Gene
>Troides_aeacus_Exon_4_CM067271.1_7702074_7702327_left_2_right_0
GTAGTAAAGATTCTTCACCAAATTTGACCTCCGATCCTACATCTGATTCATCGGACACTAGTGGAAGTGGGAAAAGAAACGTCGATCCCAGCTCTATGAATAATACCTGGGATATGTCAGCAAGAAACAACGGTAATAAGAATAGTAGCCAATCATTTCAAACGCCCCGCGAAGAGGACCCGAAGACGactaaaatgtatgaaaatactAATTGTGAGGTTACTAATAAGGAAGCCACTGTGAAAACTGGCGAA
7701704 7701972
0 2
Getting Gene
>Troides_aeacus_Exon_5_CM067271.1_7701704_7701972_left_0_right_2
GAGTCCCTGCCTAGCACTAAAGAAAACAATCAGCCAAATAGTGACAAGAAACTTGTAATATCTGAAAATTCAGATTGCGGTTATGGTACACAAATTGAAAATCAAGGATCTATTTCGACTTCAAGTAATGAAGACGAGTTACCAAAAAAAAAGCCAGTACATCAAAAACCTCATACATCCAAACAGAGAATAAATAACAAGCCCCGTTCCAGTGTTACCATTCCCGAGAGAAAAAGAAAGAGAATCGTTAAAAGGGGAAAAACTAACAT
7701274 7701628
1 0
Getting Gene
>Troides_aeacus_Exon_6_CM067271.1_7701274_7701628_left_1_right_0
aattacACAACAAGGACACAGCACGCTAACAGACGATGACATTTCCAATATCCTTAAAGAGTTTACAGTAGATTTTCTGCTGAAGGGATACAACTCGCTGGTACAGACTTTGCATACCGAGATCCTTACAAATGTTCAGCTAGAAATAGATACGTCGCATTTTTTTTGGCTGGTTACGTAC

6589314 6589473
2 2
Getting Gene
>Troides_oblongomaculatus_Exon_13_JARDWC010000431.1_6589314_6589473_left_2_right_2
cAAAACTAAAATTCTCCATGAGCGAAGTAATGGAGAGTAGTGCACTAAATTCAAAGCAAATGAACCCTCCAAAAGACGTTATAATTACAAACtctattgataatttttatcaaattcACAAACAGAATAACCTAGGAACTAACGTCAATTTCACGCCGAT
6589549 6589653
1 2
Getting Gene
>Troides_oblongomaculatus_Exon_14_JARDWC010000431.1_6589549_6589653_left_1_right_2
GCCTAGTTCATCATTTCACTCAACCGTCGGAGAAGTGCAAAAGCATAATTGGCTTGAGATTGTCCAGAAATCTCAAGAATATAGGATATTTTTTGATAAAGGAAG
6591853 6592041
1 2
Getting Gene
>Troides_oblongomaculatus_Exon_15_JARDWC010000431.1_6591853_6592041_left_1_right_2
CTCAACACTTAAAAAGGAAATCGTAGAGACGACGGTTAGAAAAACAGAGGAGATAAAAAAGGTTGAAGAAGTTCCTTCCATGACGGATGTTGCGACTCTCATTTTGCCTACAGTGATGAGCGTGAAGCAAGAGACAAGCGCTACCTCATCTAACCTCCATCTTATCGAGcctgaatattataataacag
6592136 6592325
1 0
Getting Gene
TGTATGTGAAACGGCTTCAGTGGCTTCGGATCTAACACGGATGTACGTGTCCGACGAGGACGAGAAACCCGAAATACCATTACGCTCACTGGCAGCAGTGAAAACCCATATTGACGAGCATTCCTCTAGCGACACCGATAGTTTCTTAGCCGAAAAACGTCC